In [ ]:
###########################################################################################################
# 011b_read_new_sum_duckdb.ipynb
#
# Union the smaller table of Amazon Reviews from step 3b (10x) as a view 
#       then:
#           1) Summary 1
#               a. Create a new column, year from date_time
#               b. Filter after 2018
#               c. Group by year
#               d. Summarize by row count and mean rating
#               e. Arrange by year (implicit)
#           2) Summary 2
#               a. Create a new column, year from date_time
#               b. Filter after 2018
#               c. Group by year and helpful
#               d. Summarize by row count and mean rating
#               e. Arrange by year and helpful (implicit)
#           3) Summary 3
#               a. Create a new column, year from date_time
#               b. Filter after 2018
#               c. Group by year, helpful, and verified_purchase
#               d. Summarize by row count and mean rating
#               e. Arrange by year, helpful, and verified_purchase (implicit)
#
# Before running:
#   1) Change the core_path variable
#
# Stephen McDaniel at https://PeakPython.com
# 2024-NOV-19
#
# License: MIT License
###########################################################################################################

core_path = '/root/pc_01_pandasibis/110_McAuley_Amazon_Data/'

In [1]:
# Start of notebook
import time
notebook_start_time = time.time()


In [ ]:
# import pandas as pd

import ibis

import ibis.selectors as s
from ibis import _


ibis.options.interactive = True

#from itables import show
#from itables.javascript import init_notebook_mode

# Initialize itables
#init_notebook_mode(all_interactive=True)

In [ ]:

# Connect to the DuckDB file
con = ibis.duckdb.connect(core_path + "/duckdb/db.duckdb")

In [ ]:
con.list_tables()

['All_Beauty',
 'Health_and_Household',
 'Health_and_Household_1',
 'Health_and_Household_10',
 'Health_and_Household_2']

In [5]:

# Assume an existing table in DuckDB named 'original_table'
# Reference the table using Ibis

con.raw_sql("""
    CREATE OR REPLACE VIEW Health_and_Household_10 AS
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    UNION ALL
    SELECT * FROM Health_and_Household_1
    ;
""")


t0 = con.table("Health_and_Household_10")

# Define columns for the new tables
#columns_table1 = ['row_number', 'rating', 'asin', 'parent_asin', 'user_id', 'helpful_vote', 'verified_purchase', 'date_time']
#columns_table2 = ['row_number', 'title', 'text']


In [6]:
t0.aggregate(n=t0.count()/1000000)

┏━━━━━━━━━┓
┃ n       ┃
┡━━━━━━━━━┩
│ float64 │
├─────────┤
│   100.0 │
└─────────┘

In [9]:
sum_df =(
    t0
        .mutate(y1 = t0.date_time.year())
)

sum_df = (
    sum_df
        .filter(_.y1 >= 2018)
        .group_by('y1')
        .aggregate(
            n=_.count(),
            mean = _.rating.mean()
        )
        .execute()
)

sum_df

,y1,n,mean
0,2018,9428990,4.292539
1,2019,11459740,4.360489
2,2020,15245690,4.293249
3,2021,15378010,4.206207
4,2022,14847040,4.168683
5,2023,3847270,4.199349


In [10]:
t=t0

In [11]:
sum_df =(
    t
        .mutate(year = t.date_time.year(), helpful = t.helpful_vote > 5)
        .filter(_.year >= 2018)
        .group_by('year', 'helpful')
        .aggregate(
            n=_.count(),
            mean = _.rating.mean()
        )        
        .order_by('year', 'helpful')
        .execute()
)

sum_df

,year,helpful,n,mean
0,2018,False,8922230,4.312791
1,2018,True,506760,3.935985
2,2019,False,10927750,4.378992
3,2019,True,531990,3.980413
4,2020,False,14697180,4.307364
5,2020,True,548510,3.915061
6,2021,False,14958560,4.213825
7,2021,True,419450,3.934557
8,2022,False,14469580,4.166776
9,2022,True,377460,4.241800


In [12]:
sum_df =(
    t
        .mutate(year = t.date_time.year(), helpful = t.helpful_vote > 5)
        .filter(_.year >= 2018)
        .group_by('year', 'helpful', 'verified_purchase')
        .aggregate(
            n=_.count(),
            mean = _.rating.mean()
        )
        .order_by('year', 'helpful', 'verified_purchase')
        .execute()
)

sum_df

,year,helpful,verified_purchase,n,mean
0,2018,False,False,555280,4.042771
1,2018,False,True,8366950,4.330711
2,2018,True,False,17770,3.449071
3,2018,True,True,488990,3.953680
4,2019,False,False,577410,4.074020
5,2019,False,True,10350340,4.396005
6,2019,True,False,17970,3.599332
7,2019,True,True,514020,3.993736
8,2020,False,False,771940,4.128235
9,2020,False,True,13925240,4.317294


In [13]:
con.disconnect()

In [14]:
# End of notebook
notebook_end_time = time.time()
runtime_seconds = notebook_end_time - notebook_start_time
print(f"Total notebook runtime: {runtime_seconds:.2f} seconds")


Total notebook runtime: 5.57 seconds
